## Varrer &amp; Interligar, Revisitado

Anteriormente nesse notebook vimos um exemplo de _varrer &amp; interligar_: usando um histograma de consulta dinâmica para destacar pontos num gráfico de dispersão de filmes. Aqui visitaremos alguns exemplos adicionais envolvendo seleções interligadas.

Voltando ao dataset `cars`, podemos usar o operador `repeat` para construir uma [Matriz de gráficos de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência em cavalos. Podemos definir uma seleção `interval` e inclui-la _dentro_ da nossa especificação repetida de gráficos de dispersão para habilitar seleções ligadas dentre todos os gráficos.

_Clique e arraste em qualquer um dos gráficos abaixo para varrer &amp; interligar!_

In [ ]:
brush = alt.selection_interval(
    resolve='global' # resolve all selections to a single global instance
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

Note acima o uso de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que dentre todos os gráficos apenas uma varredura pode estar ativa por vez. Entretanto, em alguns casos nos podemos querer definir varreduras em vários gráficos e juntar os resultados. Se usarmos `resolve='union'`, a seleção será a _união_ de todas as varreduras: se um ponto está dentro de qualquer área varrida será selecionado. Em contraste com o uso de `resolve='intersect'`, a seleção vai consistirá na _interseção_ de todas as varreduras: apenas pontos que estão dentro de todas as áreas varridas serão selecionados.

_Tente colocar o parâmetro `resolve` para `'union'` e para `'intersect'` e veja como se altera a lógica de seleção._

### Filtragem Cruzada

Os exemplos sobre varre &amp; interligar que vimos usam codificações condicionais, por exemplo para mudar os valores da opacidade em resposta a uma seleção. Outra opção é usar a seleção definida em um gráfico para _filtrar_ o conteúdo de outro gráfico.

Vamos construir uma coleção de histogramas para o dataset `flights`: atraso `delay` (o quão cedo ou o quão tarde o voo pousou, em minutos), distância percorrida (em milhas) `distance` e horário de decolagem `time` (horário do dia). Usaremos o operador `repeat` para criar os histogramas e adicionar uma seleção `interval` para o eixo `x` com varreduras que funcionam por interseção.

Em particular, cada histograma consistirá de duas camadas: uma camada de fundo cinza e uma camada superior azul, com a camada superior filtrada pela nossa seleção de varreduras por interseção. O resultado é uma interação de _filtragem cruzada_ com os três gráficos!

_Use as varreduras arrastando nos gráficos abaixo. Ao selecionar voos com atrasos maiores ou menores, como a distribuição da distância percorrida e do horário de decolagem se comportam?_

In [ ]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # up to 100 bins
        axis=alt.Axis(format='d', titleAnchor='start') # integer format, left-aligned title
    ),
    alt.Y('count():Q', title=None) # no y-axis title
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # clamp delays > 3 hours
    time='hours(datum.date) + minutes(datum.date) / 60' # fractional hours
).configure_view(
    stroke='transparent' # no outline
)

_Usando a filtragem cruzada você pode observar que voos atrasados tem mais chance de terem decolado mais tarde no dia. Esse fenômeno é comum para viajantes frequentes: o atraso pode se propagar ao longo do dia, afetando voos subsequentes daquele mesmo avião. Para chegar na hora com chances melhores, voe cedo!_

A combinação de vários gráficos e seleções interativas pode possibilitar formas valiosas de entendimento multidimensional, tornando até histogramas básicos em aparatos de entrada poderosos para fazer perguntas sobre um dataset!